In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
train_df.head()

In [ ]:
test_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')
test_df.head()

In [ ]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df)

In [ ]:
print(train_ds)

In [ ]:
from transformers import AutoTokenizer

model_dir = '/kaggle/input/huggingface-bert/bert-large-cased'
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [ ]:
choices = 'ABCDE'
indices = list(range(5))

choice_to_index = {choice: index for choice, index in zip(choices, indices)}
index_to_choice = {index: choice for choice, index in zip(choices, indices)}

In [ ]:
def preprocess(sample):
    
    first_sentence = [sample['prompt']] * 5
    second_sentence = []
    
    for choice in choices:
        second_sentence.append(sample[choice])
    
    tokenized_batch = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_batch['label'] = choice_to_index[sample['answer']]
    
    return tokenized_batch

In [ ]:
tokenized_train_data = train_ds.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'], batched=False)

In [ ]:
from dataclasses import dataclass
from typing import Optional, Union
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
import torch

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
from transformers import AutoModelForMultipleChoice

model = AutoModelForMultipleChoice.from_pretrained(model_dir)

In [ ]:
learning_rate = 2e-5

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

model_dir = "fine-tuned-bert"
training_args = TrainingArguments(
    output_dir=model_dir,
    learning_rate=learning_rate,
    lr_scheduler_type="linear",
    optim="adafactor",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=8,
    load_best_model_at_end=True,
    report_to='none',
)

In [ ]:
early_stopping = EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.001)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_train_data,
    tokenizer=tokenizer,
    callbacks=[early_stopping],
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
)

In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_train_data)

In [ ]:
def predictions_to_map_output(predictions):
    sorted_answer_indices = np.argsort(-predictions)
    top_answer_indices = sorted_answer_indices[:,:3]
    top_answers = np.vectorize(index_to_choice.get)(top_answer_indices)
    return np.apply_along_axis(lambda row: ' '.join(row), 1, top_answers)

In [ ]:
test_df['answer'] = 'A'
test_ds = Dataset.from_pandas(test_df)
tokenized_test_data = test_ds.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E'], batched=False)

In [ ]:
test_predictions = trainer.predict(tokenized_test_data)

In [ ]:
submission_df = test_df[['id']]
submission_df['prediction'] = predictions_to_map_output(test_predictions.predictions)

submission_df.head()

In [ ]:
submission_df.to_csv('submission.csv', index=False, header=True)